In [ ]:
import os
import numpy as np
# import scipy as 
import pandas as pd
import pandapower as pp

In [ ]:
def ReadData(folder_name):
	"""
	Return the network data as a dictionary: {key=name, data=pandas dataframe}

	:param kind: folder of the different files
	:type kind: str
	:raise lumache.InvalidKindError: If the kind is invalid.
	:return: dictionary with the data
	:rtype: dict

	"""
	data = {}
	extensions = ['txt', 'csv']
	folder_path = os.path.join('../Data',folder_name)
	# iterate through all file
	for file in os.listdir(folder_path):
		#File expected as "name_file.file_extension"
		file_name, file_extension = file.split('.') 
		if file_extension in extensions:
			file_path = os.path.join(folder_path, file)
			file_data = pd.read_csv(file_path)
			data[file_name] = file_data
	return data

def ReadTimeseries(path_timeseries_data):
	"""
	Return a 

	:param kind: Optional "kind" of ingredients.
	:type kind: list[str] or None
	:raise lumache.InvalidKindError: If the kind is invalid.
	:return: The ingredients list.
	:rtype: list[str]

	"""
	data = ''
	return data

def BuildNetwork(network_data):
	"""
	Return a 

	:param kind: Optional "kind" of ingredients.
	:type kind: list[str] or None
	:raise lumache.InvalidKindError: If the kind is invalid.
	:return: The ingredients list.
	:rtype: list[str]

	"""

	net = pp.create_empty_network()
	for elements,data in network_data.items():
		if(elements=='buses'):
			choosable = []
			for b in data.itertuples(index=False):
				id_bus = pp.create_bus(net, name = b[0], vn_kv = b[1], type = b[2])
				choosable.append(id_bus%2) #TODO: find a more meaningful way to find the choosable buses
			net.bus['choosable'] = choosable
		elif(elements=='lines'):
			for l in data.itertuples(index=False):
				pp.create_line(net, name = l[0], from_bus = l[1], to_bus = l[2], length_km = l[3], std_type = l[4])
		elif(elements=='transformers'):
			for t in data.itertuples(index=False):
				pp.create_transformer_from_parameters(net, hv_bus=t[0], lv_bus=t[1], i0_percent=t[2], pfe_kw=t[3],
										vkr_percent=t[4], sn_mva=t[5], vn_lv_kv=t[6], vn_hv_kv=t[7], vk_percent=t[8])
		elif(elements=='loads'):
			for l in data.itertuples(index=False):
				pp.create_load(net, l[0], p_mw = l[1], q_mvar = l[2], name = l[3])
	pp.create_ext_grid(net,0)
	return net

In [ ]:
#Scenarios

#Nice example for PV hosting capacity
#https://github.com/e2nIEE/pandapower/blob/develop/tutorials/hosting_capacity.ipynb
def Chose_buses(net, percentuage):
	#TODO: can be more complex: you may want to exclude some buses (ext-grid bus, ...), attach a PV only where load is, ...
	choosable_buses = net.bus[ net.bus['choosable'] == 1]
	elements_to_select = percentuage if percentuage>=1 else round(len(choosable_buses)*percentuage) #fixed number (1, 4, ...) or percentuages of the total
	return np.random.choice(net.bus.index, elements_to_select, replace=False)

def PVscenario(net, percentuage):
	new_net = net.deepcopy() #TODO Deepcopy solve the problems but there may be memory leaks. [old]This function return the original element (net) modified not a new element. Take attention
	ids = Chose_buses(new_net, percentuage)
	p_mw = 1 #TODO: to be defined better. The active generation of the sgen
	for i in ids:
		pp.create_sgen(new_net, i, p_mw=p_mw, q_mvar=0)
	return new_net
	

In [ ]:
#Network Validation
def IssueDetection(net): #Issue detection 
	#TODO: change default values.
	#TODO: many problems could happen so you may not want to have an if-else and not id but ids.
	
	# if net.res_line.loading_percent.max() > 50:
	# 	line_id = net.res_line.loading_percent.argmax()
	# 	return (True, "Line \n Overloading")
	# elif net.res_trafo.loading_percent.max() > 50:
	# 	trafo_id = net.res_trafo.loading_percent.argmax()
	# 	return (True, "Transformer \n Overloading")
	if net.res_bus.vm_pu.max() > 1.05:
		bus_id = net.res_bus.vm_pu.argmax()
		return (True, "Voltage \n Violation")
	else:
		return (False, None)

In [ ]:
#Main

#initialize variable to be used
scenarios = ['PV', 'EV'] #TODO: add others cases
scenario = 'PV'

n_simulation_per_scenario = 1 #TODO: it can be a list [50, 1000, ...]

issues, positions, times = [],[],[] #TODO: can be a more complex stucture (class, ...)


In [ ]:
#Read network data
path_network_data = 'Test'
network_data = ReadData(path_network_data)

In [ ]:
#Read timeseries
path_timeseries_data = ''
timeseries = ReadTimeseries(path_timeseries_data) #TODO: define timeseries
timeseries = [1,2] #temp values

In [ ]:
#Build the network
net = BuildNetwork(network_data)

In [ ]:
new_net = PVscenario(net,1)

In [ ]:
from pandapower.plotting.plotly import simple_plotly
new_net = PVscenario(net,1)
plot = simple_plotly(net)
plot = simple_plotly(new_net)

In [ ]:
for n in range(n_simulation_per_scenario):
	for t in timeseries:
		# net = BuildNetwork(network_data)
		if(scenario == scenarios[0]):
			new_net = PVscenario(net,1)
		else:
			a = 'blabla' #TODO: to add the different cases implementation
		pp.runpp(new_net)
		issues = NetworkValidation(new_net)
		break
